In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.utils.data import Subset
from sklearn.metrics import accuracy_score

# Define transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load SVHN dataset
trainset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform)
testset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform)

# Create a subset of the dataset (25%)
subset_indices = torch.randperm(len(trainset))[:len(trainset)//4]
trainset_subset = Subset(trainset, subset_indices)

# Define dataloaders
trainloader = torch.utils.data.DataLoader(trainset_subset, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

# Function to define model, loss function, and optimizer
def initialize_model(model_name, num_classes):
    if model_name == 'LeNet':
        model = models.LeNet()
    elif model_name == 'AlexNet':
        model = models.alexnet()
    elif model_name == 'VGG':
        model = models.vgg16()
    elif model_name == 'ResNet18':
        model = models.resnet18()
    elif model_name == 'ResNet50':
        model = models.resnet50()
    elif model_name == 'ResNet101':
        model = models.resnet101()
    else:
        raise ValueError("Unsupported model architecture")

    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)

    return model

# Function to train the model
def train_model(model, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in trainloader:
            inputs, labels = inputs.cuda(), labels.cuda()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainloader)}")

# Function to evaluate the model
def evaluate_model(model):
    model.eval()
    predictions, true_labels = [], []
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, predictions)
    print(f"Test Accuracy: {accuracy}")


100%|██████████| 182040794/182040794 [00:12<00:00, 14447600.16it/s]


100%|██████████| 64275384/64275384 [00:05<00:00, 11526035.96it/s]


ResNet50

In [4]:
model_name = 'ResNet50'
num_classes = 10
model = initialize_model(model_name, num_classes).cuda()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
train_model(model, criterion, optimizer, num_epochs=2)

# Evaluate the model
evaluate_model(model)

Epoch 1/2, Loss: 2.5852946234415133
Epoch 2/2, Loss: 2.273663561797683
Test Accuracy: 0.30954210202827287


ResNet101

In [5]:
model_name = 'ResNet101'
num_classes = 10
model = initialize_model(model_name, num_classes).cuda()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
train_model(model, criterion, optimizer, num_epochs=2)

# Evaluate the model
evaluate_model(model)

Epoch 1/2, Loss: 2.5944548518661845
Epoch 2/2, Loss: 2.34894856420487
Test Accuracy: 0.27178088506453596


ResNet18

In [6]:
model_name = 'ResNet18'
num_classes = 10
model = initialize_model(model_name, num_classes).cuda()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
train_model(model, criterion, optimizer, num_epochs=2)

# Evaluate the model
evaluate_model(model)

Epoch 1/2, Loss: 1.6936025667356986
Epoch 2/2, Loss: 0.7759212194624996
Test Accuracy: 0.770858942839582


LeNet

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LeNet5(nn.Module):
    def __init__(self, num_classes=10, input_channels=3):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def initialize_model(model_name, num_classes, input_channels=3):
    if model_name == 'LeNet-5':
        return LeNet5(num_classes, input_channels)
    else:
        raise ValueError("Unsupported model architecture")

# Now you can use the provided code with LeNet-5 model
model_name = 'LeNet-5'
num_classes = 10
input_channels = 3  # For RGB images
model = initialize_model(model_name, num_classes, input_channels).cuda()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
train_model(model, criterion, optimizer, num_epochs=2)

# Evaluate the model
evaluate_model(model)


Epoch 1/2, Loss: 2.2609799573974874
Epoch 2/2, Loss: 2.236764294106715
Test Accuracy: 0.1958743085433313


AlexNet

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Define transforms for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load the SVHN dataset
full_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)

# Split the dataset into training and testing sets
train_indices, test_indices = train_test_split(range(len(full_dataset)), test_size=0.75, random_state=42)

# Define data loaders for the reduced dataset
train_loader = DataLoader(full_dataset, batch_size=64, sampler=SubsetRandomSampler(train_indices))
test_loader = DataLoader(full_dataset, batch_size=64, sampler=SubsetRandomSampler(test_indices))

# Define the AlexNet model
class AlexNet(nn.Module):
    def __init__(self, num_classes=10):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Initialize the model, loss function, and optimizer
model = AlexNet(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
for epoch in range(2):  # Number of epochs
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # Print every 100 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

# Evaluation
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print('Accuracy on test set: %.2f%%' % (accuracy * 100))


Using downloaded and verified file: ./data/train_32x32.mat
[1,   100] loss: 2.262
[1,   200] loss: 2.279
[2,   100] loss: 2.266
[2,   200] loss: 2.269
Finished Training
Accuracy on test set: 18.79%
